# 0. 개요

- 모든 시간대를 대상으로 하는 것이 아닌, 예측해야하는 시간대의 target이 비슷한 데이터를 바탕으로
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
- 모델을 제작

# 1. 라이브러리 로드 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import time
import datetime
import os

from mypackage import *

pd.set_option('display.max_columns', None)

### <div style="color:red">파일명 체크!!!</div>

In [ ]:
print('data loading')
with open('inputs/all_df2.pickle', 'rb') as f:
    all_df = pickle.load(f)
print('data loaded')

# 2. 데이터 탐색

여기서는 test셋과 비슷한 시간대를 찾아서, 이 데이터만을 바탕으로 모델을 만들것임.  

test: 2016-04-25 ~ 2016-05-22  17~20주차 (16주차나 21주차까지 늘려야될지도 생각해보자)

---
train: 2015-04-27 ~ 2015-05-24  
train: 2014-04-28 ~ 2014-05-25  / 2014-04-21 ~ 2014-05-18
train: 2013-04-27 ~ 2013-05-24  
train: 2012-04-27 ~ 2012-05-24 
train: 2011-04-27 ~ 2011-05-24 

In [ ]:
c00 = all_df['week'] == 12
c01 = all_df['week'] == 11
c02 = all_df['week'] == 10
c03 = all_df['week'] == 19

c1 = all_df['week'] == 13
c2 = all_df['week'] == 14
c3 = all_df['week'] == 15
c4 = all_df['week'] == 16

c5 = all_df['week'] == 17
c6 = all_df['week'] == 18
c7 = all_df['week'] == 19
c8 = all_df['week'] == 20
all_df17_20 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8| c00| c01| c02| c03]

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

# 4. 모델 제작

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
# rf.fit(train2, np.log(train['target'] + 1))
# result = rf.predict(test2)

In [ ]:
# from lightgbm import LGBMRegressor
# # lgb = LGBMRegressor(num_leaves=2000, colsample_bytree=0.6, subsample=0.6, n_estimators=600, learning_rate=0.025, n_jobs=-1, device='gpu', max_bin = 63)
# lgb = LGBMRegressor(num_leaves=20, colsample_bytree=0.6, subsample=0.6, n_estimators=60, learning_rate=0.02, n_jobs=-1, device='cpu')

# lgb.fit(train, target)
# result = lgb.predict(test)

## 4.1 train set /  test set 선정

feature 선정에 있어서 다른 노트북에서는 all_df = all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1) 이렇게 진행했음

In [ ]:
import lightgbm as lgb
import gc

# features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'weekday', 'event_name_1', 'event_type_1',  
#             'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
#             'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
#           'rolling_mean_t60', 'rolling_std_t60']

features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'weekday',
            'event_name_1', 'event_type_1',  'snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
            'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28', 'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
            'rolling_mean_t84', 'rolling_mean_t168', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t28']


train_set_X = all_df17_20[all_df17_20['date'] <= '2016-04-24']
train_set_y = train_set_X['target']

train_set_X = train_set_X[features]




# 나중에 합칠 때 필요해서 test에 선언
test = all_df17_20[all_df17_20['date'] > '2016-04-24']

# 테스트 셋
test_set = all_df17_20[all_df17_20['date'] > '2016-04-24']
test_set = test_set[features]

# del all_df

## 4.2 KFold - LGBM 모델

In [ ]:
n_fold = 5
folds = KFold(n_splits=5, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []


for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 400,
        learning_rate = 0.1,
        n_jobs = -1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['target'] = y_preds



In [ ]:
feature_importances

# 5. 예측 및 제출

In [ ]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'target']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'target').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final.to_csv('submissions/submission.csv', index = False)

In [ ]:
final.head()

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [ ]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

# 6. 모델 파라미터 및 피처 기록 및 모델 저장하기

In [ ]:
write_params_features(features, params, eval_results)

In [ ]:
save_feature_importance(feature_importances)